# Pembangunan Model Leksikal Langsung - Sistem Pengenal Emosi Cakapan

## Library Preparation

In [1]:
!pip install imbalanced-learn
!pip install transformers
!pip install sentence_transformers

In [2]:
import os

import numpy as np
import pandas as pd
import statistics
import joblib
import torch

import nltk
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, AutoModel

import gensim
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, SentenceTransformer
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras import layers
from imblearn.over_sampling import SMOTE

from google.colab import drive
from google.colab import files

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
drive.mount('/content/gdrive')
drive_dir = "/content/gdrive/MyDrive/Teknik Informatika/Semester 7/TUGAS AKHIR/TA 2"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
print(tf.__version__)

2.5.0


## Feature Extraction

In [5]:
data = joblib.load(drive_dir + '/Eksperimen/Data/basic.pkl')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10822 entries, 0 to 10821
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID Ucapan        10822 non-null  object
 1   Transkripsi      10822 non-null  object
 2   Label Emosi      10822 non-null  object
 3   Abstraksi Emosi  2003 non-null   object
dtypes: object(4)
memory usage: 338.3+ KB


### TFIDF

In [6]:
tfidf_vectorizer = TfidfVectorizer()
feature_tfidf = tfidf_vectorizer.fit_transform(data['Transkripsi'])
feature_tfidf = feature_tfidf.todense()
feature_tfidf.shape

(10822, 8304)

### Average Word2Vec

In [7]:
def w2v_embedding(data, model):
    matrix_data = []
    for text in data :
        matrix = []
        for word in text.split() :
            try :
                matrix.append(model.wv[word])
            except :
                matrix.append(np.zeros((300,), dtype=float))
        matrix = np.array(matrix)
        matrix_data.append((matrix.mean(axis=0)))   # Average word2vec
        # matrix_data.append(matrix)   # Concatenate vectors word2vec
    return np.array(matrix_data)

In [8]:
path = drive_dir + '/Eksperimen/Pretrained Model/Word2Vec/idwiki_word2vec_300.model'
w2v_model = gensim.models.word2vec.Word2Vec.load(path)

In [9]:
feature_w2v = w2v_embedding(data['Transkripsi'], w2v_model)   
feature_w2v = tf.keras.preprocessing.sequence.pad_sequences(feature_w2v, padding="post", dtype='float32')
feature_w2v.shape

(10822, 300)

### IndoBERT

In [10]:
# load pretrained model/tokenizer
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = AutoModel.from_pretrained("indobenchmark/indobert-base-p1")

In [11]:
# prepare data

tokenized = data['Transkripsi'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized.values: 
  if len(i) > max_len: max_len = len(i)
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [ ]:
# modelling
input_ids = torch.LongTensor(padded)
attention_mask = torch.LongTensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
feature_bert = last_hidden_states[0][:,0,:].numpy()
feature_bert.shape

## Data Preparation

### Construct Sequential Data 

In [7]:
def construct_sequential(id, data):
  sequential_datas = []
  new_data= []
  for i in range(len(id)):
    if id[i].endswith('001'):
      sequential_datas.append(new_data)
      new_data = []
    new_data.append(data[i])
  sequential_datas.append(new_data)

  return np.array(sequential_datas, dtype=object)[1:]

In [8]:
def construct_abstract(id, abstraksi):
  new_abstraksi = []
  for i in range(len(id)):
    if id[i].endswith('001'):
      new_abstraksi.append(abstraksi[i])

  return np.array(new_abstraksi, dtype=object)

In [9]:
id = data['ID Ucapan']
sequential_features_tfidf, sequential_features_w2v, sequential_features_bert = construct_sequential(id, feature_tfidf), construct_sequential(id, feature_tfidf), construct_sequential(id, feature_tfidf)
print(sequential_features_tfidf.shape, sequential_features_w2v.shape, sequential_features_bert.shape)

(2003,) (2003,) (2003,)


In [10]:
abstraksi = construct_abstract(id, data['Abstraksi Emosi'])
print(abstraksi.shape)

(2003,)


### Label One Hot Encoding

In [11]:
encoder_cakapan = OneHotEncoder(sparse=False)
abstraksi = abstraksi.reshape(-1, 1)
abstraksi = encoder_cakapan.fit_transform(abstraksi)
abstraksi.shape

(2003, 6)

### Split Train and Test Data

In [12]:
X_tfidf = sequential_features_tfidf
X_w2v = sequential_features_w2v
X_bert = sequential_features_bert
y = abstraksi

In [13]:
X_tfidf = tf.keras.preprocessing.sequence.pad_sequences(X_tfidf, padding="post", dtype='float32')
X_w2v = tf.keras.preprocessing.sequence.pad_sequences(X_w2v, padding="post", dtype='float32')
X_bert = tf.keras.preprocessing.sequence.pad_sequences(X_bert, padding="post", dtype='float32')
X_tfidf = X_tfidf.reshape(2003, 15, 8304)
y = tf.keras.preprocessing.sequence.pad_sequences(y, padding="post", dtype='float32')
print("TFIDF", X_tfidf.shape, "W2V", X_w2v.shape, "BERT", X_bert.shape)
print("Abstract", y.shape)

TFIDF (2003, 15, 8304) W2V (2003, 15, 1, 8304) BERT (2003, 15, 1, 8304)
Abstract (2003, 6)


In [14]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
for train_index, test_index in sss.split(X_tfidf, y):
  train_index, test_index = train_index, test_index
print("Train data", train_index.shape)
print("Test data", test_index.shape)

Train data (1702,)
Test data (301,)


## Modelling Preparation

### Define Model

In [15]:
def get_lstm_model(shape, label_shape):
  model = Sequential()
  model.add(layers.Masking(mask_value=0, input_shape=shape))
  model.add(layers.LSTM(32))
  model.add(layers.Dense(label_shape, activation='softmax'))
  return model

In [16]:
def get_gru_model(shape, label_shape):
  model = Sequential()
  model.add(layers.Masking(mask_value=0, input_shape=shape))
  model.add(layers.GRU(32))
  model.add(layers.Dense(label_shape, activation='softmax'))
  return model

In [17]:
def get_model_1(shape, label_shape):
  model = Sequential()
  model.add(layers.Masking(mask_value=0, input_shape=shape))
  model.add(layers.GRU(32))
  model.add(layers.Dense(label_shape, activation='softmax'))
  return model

In [18]:
def get_model_2(shape, label_shape):
  model = Sequential()
  model.add(layers.Masking(mask_value=0, input_shape=shape))
  model.add(layers.GRU(32))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(label_shape, activation='softmax'))
  return model

In [19]:
def get_model_3(shape, label_shape):
  model = Sequential()
  model.add(layers.Masking(mask_value=0, input_shape=shape))
  model.add(layers.GRU(32))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(label_shape, activation='softmax'))
  return model

In [20]:
def create_model(shape, label_shape, mode):
  model = None
  if mode == 1:
    model = get_model_1(shape, label_shape)
  elif mode == 2 :
    model = get_model_2(shape, label_shape)
  elif mode == 3 :
    model = get_model_3(shape, label_shape)
  elif mode == "lstm":
    model = get_lstm_model(shape, label_shape)
  elif mode == "gru":
    model = get_gru_model(shape, label_shape)
  return model

### Define Training Parameter

In [21]:
batch_size = 10
epochs = 10
validation_split = 0.2
optimizer = 'rmsprop'
loss = 'categorical_crossentropy'
metrics = ['accuracy']
callbacks = [EarlyStopping(monitor='val_loss', mode='min', min_delta=0.0001, patience=1, restore_best_weights=True)]

### Define Schema Experiment

In [22]:
def train_5fold_cv(X, y, shape, label_shape, mode):

  loss_score = []
  accuracy_score = []  

  cv = KFold(n_splits=5, random_state=42, shuffle=True)

  for train_index, val_index in cv.split(X):
      
      X_train, X_val, y_train, y_val = X[train_index], X[val_index], y[train_index], y[val_index]
      
      model = create_model(shape, label_shape, mode)
      model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
      model.fit(X_train, y_train,
              validation_data=(X_val, y_val),
              epochs=epochs,
              batch_size=batch_size,
              callbacks=callbacks)
      
      l, a = model.evaluate(X_val, y_val)
      loss_score.append(l)
      accuracy_score.append(a)
  
  loss_score = np.round(np.mean(loss_score), 4)
  accuracy_score = np.round(np.mean(accuracy_score), 4)
  print("Loss Score : ", loss_score, "Accuracy Score : ", accuracy_score)
  return loss_score, accuracy_score

In [23]:
def train_stratified_random_sampling(X_train, y_train, X_test, y_test, shape, label_shape, mode, encoder):

  model = create_model(shape, label_shape, mode)
  model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
  model.fit(X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks)
      
  loss_score, accuracy_score = model.evaluate(X_test, y_test)
  loss_score = np.round(np.mean(loss_score), 4)
  accuracy_score = np.round(np.mean(accuracy_score), 4)

  pred = model.predict(X_test)
  y_pred = encoder.inverse_transform(pred)
  y_true = encoder.inverse_transform(y_test)

  print("Loss Score : ", loss_score, "Accuracy Score : ", accuracy_score)
  print(classification_report(y_true, y_pred, digits=4))
  print(confusion_matrix(y_true, y_pred))
  
  return model, accuracy_score

## Tahap 1 : Pencarian Fitur dan Algoritma Terbaik

In [ ]:
feature = [("TFIDF", X_tfidf), ("W2V", X_w2v), ("BERT", X_bert)]
label = y
model = ["lstm", "gru"]
n = 5

In [ ]:
def run_tahap_1(feature, label, model, train_index, n):
  performances = []
  for i in range(n):
    p = []
    for f in feature:
      X_train, y_train = f[1][train_index], label[train_index]
      for m in model:
        loss_score, accuracy_score = train_5fold_cv(X_train, y_train, X_train.shape[1:], y.shape[1], m)
        p.append((f[0] + ' ' + m, loss_score, accuracy_score))
    performances.append((i, p))
  return performances

In [ ]:
performances = run_tahap_1(feature, label, model, train_index, n)

Epoch 1/10
137/137 [==============================] - 14s 69ms/step - loss: 1.6199 - accuracy: 0.3793 - val_loss: 1.5008 - val_accuracy: 0.3431
Epoch 2/10
137/137 [==============================] - 6s 46ms/step - loss: 1.3716 - accuracy: 0.4151 - val_loss: 1.4748 - val_accuracy: 0.3636
Epoch 3/10
11/11 [==============================] - 0s 27ms/step - loss: 1.5158 - accuracy: 0.4135
Epoch 1/10
137/137 [==============================] - 11s 52ms/step - loss: 1.6411 - accuracy: 0.3939 - val_loss: 1.4918 - val_accuracy: 0.3812
Epoch 2/10
137/137 [==============================] - 6s 45ms/step - loss: 1.4095 - accuracy: 0.3679 - val_loss: 1.4311 - val_accuracy: 0.4135
Epoch 3/10
137/137 [==============================] - 6s 45ms/step - loss: 1.2721 - accuracy: 0.4483 - val_loss: 1.3723 - val_accuracy: 0.4076
Epoch 4/10
11/11 [==============================] - 0s 28ms/step - loss: 1.3819 - accuracy: 0.4516
Epoch 1/10
137/137 [==============================] - 12s 51ms/step - loss: 1.6324 - 

In [ ]:
for p in performances:
  print("Training ke- : ", p[0])
  res = pd.DataFrame(p[1], columns=['Feature + Model', 'Loss Score', 'Accuracy Score'])
  print(res)

Training ke- :  0
  Feature + Model  Loss Score  Accuracy Score
0      TFIDF lstm      1.4294          0.4377
1       TFIDF gru      1.3741          0.4513
2        W2V lstm      1.4007          0.3890
3         W2V gru      1.3636          0.4160
4       BERT lstm      1.4526          0.3919
5        BERT gru      1.4004          0.4295
Training ke- :  1
  Feature + Model  Loss Score  Accuracy Score
0      TFIDF lstm      1.4102          0.4319
1       TFIDF gru      1.3483          0.4660
2        W2V lstm      1.3947          0.3919
3         W2V gru      1.4473          0.3502
4       BERT lstm      1.5467          0.3619
5        BERT gru      1.4547          0.3972
Training ke- :  2
  Feature + Model  Loss Score  Accuracy Score
0      TFIDF lstm      1.3955          0.4478
1       TFIDF gru      1.3523          0.4419
2        W2V lstm      1.4225          0.3836
3         W2V gru      1.4029          0.3796
4       BERT lstm      1.3949          0.3932
5        BERT gru      1.3

In [ ]:
res = pd.DataFrame(performances[0][1], columns=['Feature + Model', 'Loss Score', 'Accuracy Score'])
for performance in performances[1:]:
  temp = pd.DataFrame(performance[1], columns=['Feature + Model', 'Loss Score', 'Accuracy Score'])
  res['Loss Score'] += temp['Loss Score']
  res['Accuracy Score'] += temp['Accuracy Score']
res['Loss Score'] /= n
res['Accuracy Score'] /= n
print(res.sort_values(by='Accuracy Score', ascending=False, ignore_index=True))

  Feature + Model  Loss Score  Accuracy Score
0       TFIDF gru     1.37598         0.45022
1      TFIDF lstm     1.41360         0.43540
2        BERT gru     1.43676         0.40240
3       BERT lstm     1.45106         0.38570
4         W2V gru     1.40848         0.38146
5        W2V lstm     1.40966         0.38118


## Tahap 2 : Penentuan Teknik Preprocessing

In [ ]:
preprocess = ['basic', 'normalization', 'stemming', 'stopword']
label = abstraksi
n = 5

In [ ]:
def run_tahap_2(preprocess, label, train_index, n):
  performances = []
  for i in range(n):
    p = []
    for pp in preprocess:
      data = joblib.load(drive_dir + '/Eksperimen/Data/' + pp + '.pkl')
      vectorizer = TfidfVectorizer()
      feature = vectorizer.fit_transform(data['Transkripsi'])
      sequential_feature = construct_sequential(data["ID Ucapan"], feature.todense())

      X = tf.keras.preprocessing.sequence.pad_sequences(sequential_feature, padding="post", dtype='float32')
      X = X.reshape(X.shape[0], X.shape[1], X.shape[3])

      X_train, y_train = X[train_index], label[train_index]
      
      loss_score, accuracy_score = train_5fold_cv(X_train, y_train, X_train.shape[1:], y.shape[1], 'gru')
      p.append((pp, loss_score, accuracy_score))

    performances.append((i, p))
  return performances

In [ ]:
performances = run_tahap_2(preprocess, label, model, train_index, n)

Epoch 1/10
137/137 [==============================] - 10s 48ms/step - loss: 1.5859 - accuracy: 0.3680 - val_loss: 1.4717 - val_accuracy: 0.3343
Epoch 2/10
137/137 [==============================] - 5s 38ms/step - loss: 1.3713 - accuracy: 0.3861 - val_loss: 1.4404 - val_accuracy: 0.3343
Epoch 3/10
137/137 [==============================] - 5s 38ms/step - loss: 1.2757 - accuracy: 0.4254 - val_loss: 1.4207 - val_accuracy: 0.4047
Epoch 4/10
11/11 [==============================] - 0s 24ms/step - loss: 1.5019 - accuracy: 0.4282
Epoch 1/10
137/137 [==============================] - 10s 50ms/step - loss: 1.5999 - accuracy: 0.3810 - val_loss: 1.4603 - val_accuracy: 0.3900
Epoch 2/10
137/137 [==============================] - 5s 39ms/step - loss: 1.3755 - accuracy: 0.4093 - val_loss: 1.4078 - val_accuracy: 0.4106
Epoch 3/10
137/137 [==============================] - 5s 40ms/step - loss: 1.2877 - accuracy: 0.4881 - val_loss: 1.3513 - val_accuracy: 0.4106
Epoch 4/10
11/11 [=======================

In [ ]:
for p in performances:
  print("Training ke- : ", p[0])
  res = pd.DataFrame(p[1], columns=['Preprocess', 'Loss Score', 'Accuracy Score'])
  print(res['Accuracy Score'])

Training ke- :  0
0    0.4442
1    0.4436
2    0.4495
3    0.4477
Name: Accuracy Score, dtype: float64
Training ke- :  1
0    0.4407
1    0.4454
2    0.4431
3    0.4654
Name: Accuracy Score, dtype: float64
Training ke- :  2
0    0.4530
1    0.4460
2    0.4483
3    0.4537
Name: Accuracy Score, dtype: float64
Training ke- :  3
0    0.4442
1    0.4301
2    0.4648
3    0.4360
Name: Accuracy Score, dtype: float64
Training ke- :  4
0    0.4413
1    0.4524
2    0.4624
3    0.4736
Name: Accuracy Score, dtype: float64


In [ ]:
res = pd.DataFrame(performances[0][1], columns=['Preprocess', 'Loss Score', 'Accuracy Score'])
for performance in performances[1:]:
  temp = pd.DataFrame(performance[1], columns=['Preprocess', 'Loss Score', 'Accuracy Score'])
  res['Loss Score'] += temp['Loss Score']
  res['Accuracy Score'] += temp['Accuracy Score']
res['Loss Score'] /= n
res['Accuracy Score'] /= n
print(res.sort_values(by='Accuracy Score', ascending=False, ignore_index=True))

      Preprocess  Loss Score  Accuracy Score
0       stopword     1.35268         0.45528
1       stemming     1.36138         0.45362
2          basic     1.36694         0.44468
3  normalization     1.39116         0.44350


## Tahap 3 : Pencarian Arsitektur Model Terbaik

In [ ]:
model = [1, 2, 3]
label = abstraksi
n = 5

In [ ]:
data = joblib.load(drive_dir + '/Eksperimen/Data/stopword.pkl')
vectorizer = TfidfVectorizer()
feature = vectorizer.fit_transform(data['Transkripsi'])
sequential_feature = construct_sequential(data["ID Ucapan"], feature.todense())
X = tf.keras.preprocessing.sequence.pad_sequences(sequential_feature, padding="post", dtype='float32')
X = X.reshape(X.shape[0], X.shape[1], X.shape[3])

In [ ]:
def run_tahap_3(X, label, model, train_index, n):
  performances = []
  for i in range(n):
    p = []
    X_train, y_train = X[train_index], label[train_index]
    for m in model:
      loss_score, accuracy_score = train_5fold_cv(X_train, y_train, X_train.shape[1:], y.shape[1], m)
      p.append((m, loss_score, accuracy_score))
    performances.append((i, p))
  return performances

In [ ]:
performances = run_tahap_3(X, label, model, train_index, n)

In [ ]:
for p in performances:
  print("Training ke- : ", p[0])
  res = pd.DataFrame(p[1], columns=['Architecture', 'Loss Score', 'Accuracy Score'])
  print(res['Accuracy Score'])

In [ ]:
res = pd.DataFrame(performances[0][1], columns=['Architecture', 'Loss Score', 'Accuracy Score'])
for performance in performances[1:]:
  temp = pd.DataFrame(performance[1], columns=['Architecture', 'Loss Score', 'Accuracy Score'])
  res['Loss Score'] += temp['Loss Score']
  res['Accuracy Score'] += temp['Accuracy Score']
res['Loss Score'] /= n
res['Accuracy Score'] /= n
print(res.sort_values(by='Accuracy Score', ascending=False, ignore_index=True))

## Tahap 4 : Penentuan Parameter Model

In [ ]:
data = joblib.load(drive_dir + '/Eksperimen/Data/stopword.pkl')
vectorizer = TfidfVectorizer()
feature = vectorizer.fit_transform(data['Transkripsi'])
sequential_feature = construct_sequential(data["ID Ucapan"], feature.todense())
X = tf.keras.preprocessing.sequence.pad_sequences(sequential_feature, padding="post", dtype='float32')
X = X.reshape(X.shape[0], X.shape[1], X.shape[3])
label = abstraksi

In [ ]:
n = 5

In [ ]:
def run_tahap_4(X, label, train_index, n):
  performances = []
  for i in range(n):
    p = []
    X_train, y_train = X[train_index], label[train_index]
    loss_score, accuracy_score = train_5fold_cv(X_train, y_train, X_train.shape[1:], y.shape[1], 3)
    p.append((batch_size, loss_score, accuracy_score))
    performances.append((i, p))
  return performances

### Batch Size

In [ ]:
batch_size = 10
run_tahap_4(X, label, train_index, n)

Epoch 1/10
137/137 [==============================] - 7s 19ms/step - loss: 1.6117 - accuracy: 0.3755 - val_loss: 1.4763 - val_accuracy: 0.3343
Epoch 2/10
137/137 [==============================] - 1s 9ms/step - loss: 1.3480 - accuracy: 0.3957 - val_loss: 1.4337 - val_accuracy: 0.3372
Epoch 3/10
137/137 [==============================] - 1s 9ms/step - loss: 1.2855 - accuracy: 0.4161 - val_loss: 1.4302 - val_accuracy: 0.4194
Epoch 4/10
11/11 [==============================] - 0s 7ms/step - loss: 1.4302 - accuracy: 0.4194
Epoch 1/10
137/137 [==============================] - 6s 17ms/step - loss: 1.6040 - accuracy: 0.3386 - val_loss: 1.4450 - val_accuracy: 0.3812
Epoch 2/10
137/137 [==============================] - 1s 9ms/step - loss: 1.3859 - accuracy: 0.3793 - val_loss: 1.4279 - val_accuracy: 0.3783
Epoch 3/10
137/137 [==============================] - 1s 9ms/step - loss: 1.3017 - accuracy: 0.4180 - val_loss: 1.3578 - val_accuracy: 0.4194
Epoch 4/10
11/11 [==============================

[(0, [(10, 1.3565, 0.4283)]),
 (1, [(10, 1.3035, 0.4419)]),
 (2, [(10, 1.3168, 0.4366)]),
 (3, [(10, 1.308, 0.439)]),
 (4, [(10, 1.3317, 0.4342)])]

In [ ]:
batch_size = 32
performances.append(run_tahap_4(X, label, train_index, n))

Epoch 1/10
43/43 [==============================] - 6s 42ms/step - loss: 1.6788 - accuracy: 0.3171 - val_loss: 1.5129 - val_accuracy: 0.3343
Epoch 2/10
43/43 [==============================] - 1s 17ms/step - loss: 1.4236 - accuracy: 0.3640 - val_loss: 1.4796 - val_accuracy: 0.3343
Epoch 3/10
43/43 [==============================] - 1s 15ms/step - loss: 1.3669 - accuracy: 0.3938 - val_loss: 1.4492 - val_accuracy: 0.3460
Epoch 4/10
43/43 [==============================] - 1s 14ms/step - loss: 1.2877 - accuracy: 0.4236 - val_loss: 1.4187 - val_accuracy: 0.3695
Epoch 5/10
43/43 [==============================] - 1s 15ms/step - loss: 1.1625 - accuracy: 0.5028 - val_loss: 1.3586 - val_accuracy: 0.4457
Epoch 6/10
11/11 [==============================] - 0s 7ms/step - loss: 1.3586 - accuracy: 0.4457
Epoch 1/10
43/43 [==============================] - 6s 45ms/step - loss: 1.6489 - accuracy: 0.3338 - val_loss: 1.4862 - val_accuracy: 0.3812
Epoch 2/10
43/43 [==============================] - 1s 1

In [ ]:
batch_size = 64
performances.append(run_tahap_4(X, label, train_index, n))

Epoch 1/10
22/22 [==============================] - 6s 79ms/step - loss: 1.7191 - accuracy: 0.3399 - val_loss: 1.5410 - val_accuracy: 0.3343
Epoch 2/10
22/22 [==============================] - 0s 23ms/step - loss: 1.4489 - accuracy: 0.3841 - val_loss: 1.4881 - val_accuracy: 0.3343
Epoch 3/10
22/22 [==============================] - 1s 22ms/step - loss: 1.3934 - accuracy: 0.3927 - val_loss: 1.4832 - val_accuracy: 0.3343
Epoch 4/10
22/22 [==============================] - 1s 23ms/step - loss: 1.3730 - accuracy: 0.3759 - val_loss: 1.4535 - val_accuracy: 0.3402
Epoch 5/10
22/22 [==============================] - 1s 25ms/step - loss: 1.3159 - accuracy: 0.4036 - val_loss: 1.4323 - val_accuracy: 0.3724
Epoch 6/10
11/11 [==============================] - 0s 7ms/step - loss: 1.4323 - accuracy: 0.3724
Epoch 1/10
22/22 [==============================] - 6s 78ms/step - loss: 1.7049 - accuracy: 0.3507 - val_loss: 1.5121 - val_accuracy: 0.3812
Epoch 2/10
22/22 [==============================] - 0s 2

In [ ]:
batch_size = 128
performances.append(run_tahap_4(X, label, train_index, n))

Epoch 1/10
11/11 [==============================] - 5s 157ms/step - loss: 1.7563 - accuracy: 0.3070 - val_loss: 1.6407 - val_accuracy: 0.3343
Epoch 2/10
11/11 [==============================] - 0s 40ms/step - loss: 1.5634 - accuracy: 0.3781 - val_loss: 1.5397 - val_accuracy: 0.3343
Epoch 3/10
11/11 [==============================] - 0s 40ms/step - loss: 1.4471 - accuracy: 0.3798 - val_loss: 1.4974 - val_accuracy: 0.3343
Epoch 4/10
11/11 [==============================] - 0s 33ms/step - loss: 1.3938 - accuracy: 0.3889 - val_loss: 1.4754 - val_accuracy: 0.3343
Epoch 5/10
11/11 [==============================] - 0s 37ms/step - loss: 1.3530 - accuracy: 0.4027 - val_loss: 1.4630 - val_accuracy: 0.3343
Epoch 6/10
11/11 [==============================] - 0s 37ms/step - loss: 1.3343 - accuracy: 0.3877 - val_loss: 1.4600 - val_accuracy: 0.3343
Epoch 7/10
11/11 [==============================] - 0s 39ms/step - loss: 1.2836 - accuracy: 0.3912 - val_loss: 1.4460 - val_accuracy: 0.3578
Epoch 8/10
1

### Optimizer

In [ ]:
batch_size = 32

In [ ]:
optimizer = 'adam'
run_tahap_4(X, label, train_index, n)

Epoch 1/10
43/43 [==============================] - 6s 43ms/step - loss: 1.7182 - accuracy: 0.3504 - val_loss: 1.5310 - val_accuracy: 0.3343
Epoch 2/10
43/43 [==============================] - 1s 15ms/step - loss: 1.4484 - accuracy: 0.3621 - val_loss: 1.4964 - val_accuracy: 0.3343
Epoch 3/10
43/43 [==============================] - 1s 14ms/step - loss: 1.3607 - accuracy: 0.3719 - val_loss: 1.4314 - val_accuracy: 0.3490
Epoch 4/10
43/43 [==============================] - 1s 15ms/step - loss: 1.2233 - accuracy: 0.4499 - val_loss: 1.4034 - val_accuracy: 0.4311
Epoch 5/10
11/11 [==============================] - 0s 7ms/step - loss: 1.4034 - accuracy: 0.4311
Epoch 1/10
43/43 [==============================] - 6s 52ms/step - loss: 1.7285 - accuracy: 0.3163 - val_loss: 1.5342 - val_accuracy: 0.3812
Epoch 2/10
43/43 [==============================] - 1s 14ms/step - loss: 1.4344 - accuracy: 0.3894 - val_loss: 1.4590 - val_accuracy: 0.3812
Epoch 3/10
43/43 [==============================] - 1s 1

In [ ]:
optimizer = 'rmsprop'
run_tahap_4(X, label, train_index, n)

Epoch 1/10
43/43 [==============================] - 6s 55ms/step - loss: 1.6637 - accuracy: 0.3768 - val_loss: 1.5106 - val_accuracy: 0.3343
Epoch 2/10
43/43 [==============================] - 1s 16ms/step - loss: 1.4237 - accuracy: 0.3778 - val_loss: 1.4715 - val_accuracy: 0.3343
Epoch 3/10
43/43 [==============================] - 1s 15ms/step - loss: 1.3736 - accuracy: 0.3823 - val_loss: 1.4521 - val_accuracy: 0.3343
Epoch 4/10
11/11 [==============================] - 0s 8ms/step - loss: 1.4521 - accuracy: 0.3343
Epoch 1/10
43/43 [==============================] - 5s 43ms/step - loss: 1.6853 - accuracy: 0.3221 - val_loss: 1.4769 - val_accuracy: 0.3812
Epoch 2/10
43/43 [==============================] - 1s 15ms/step - loss: 1.4373 - accuracy: 0.3790 - val_loss: 1.4743 - val_accuracy: 0.3812
Epoch 3/10
43/43 [==============================] - 1s 14ms/step - loss: 1.3574 - accuracy: 0.3984 - val_loss: 1.4349 - val_accuracy: 0.3959
Epoch 4/10
43/43 [==============================] - 1s 1

[(0, [(32, 1.3064, 0.439)]),
 (1, [(32, 1.314, 0.4348)]),
 (2, [(32, 1.3283, 0.4248)]),
 (3, [(32, 1.3055, 0.4624)]),
 (4, [(32, 1.3261, 0.4366)])]

In [ ]:
optimizer = 'sgd'
run_tahap_4(X, label, train_index, n)

Epoch 1/10
43/43 [==============================] - 6s 45ms/step - loss: 1.7698 - accuracy: 0.3210 - val_loss: 1.7143 - val_accuracy: 0.3343
Epoch 2/10
43/43 [==============================] - 1s 15ms/step - loss: 1.6879 - accuracy: 0.3654 - val_loss: 1.6511 - val_accuracy: 0.3343
Epoch 3/10
43/43 [==============================] - 1s 15ms/step - loss: 1.6076 - accuracy: 0.3888 - val_loss: 1.6062 - val_accuracy: 0.3343
Epoch 4/10
43/43 [==============================] - 1s 14ms/step - loss: 1.5632 - accuracy: 0.3841 - val_loss: 1.5746 - val_accuracy: 0.3343
Epoch 5/10
43/43 [==============================] - 1s 15ms/step - loss: 1.5304 - accuracy: 0.3879 - val_loss: 1.5526 - val_accuracy: 0.3343
Epoch 6/10
43/43 [==============================] - 1s 16ms/step - loss: 1.5140 - accuracy: 0.3784 - val_loss: 1.5379 - val_accuracy: 0.3343
Epoch 7/10
43/43 [==============================] - 1s 15ms/step - loss: 1.5143 - accuracy: 0.3566 - val_loss: 1.5274 - val_accuracy: 0.3343
Epoch 8/10
43

[(0, [(32, 1.4594, 0.3749)]),
 (1, [(32, 1.4649, 0.3749)]),
 (2, [(32, 1.4558, 0.3749)]),
 (3, [(32, 1.4599, 0.3749)]),
 (4, [(32, 1.4573, 0.3749)])]

## Tahap 5 : Penentuan Penggunaan Teknik Resampling

In [24]:
batch_size = 32
optimizer = 'adam'
callbacks = [EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.0001, patience=2, restore_best_weights=True)]

In [25]:
data = joblib.load(drive_dir + '/Eksperimen/Data/stopword.pkl')

vectorizer = TfidfVectorizer()
feature = vectorizer.fit_transform(data['Transkripsi'])
sequential_features = construct_sequential(data["ID Ucapan"], feature.todense())

X = tf.keras.preprocessing.sequence.pad_sequences(sequential_features, padding="post", dtype='float32')
X = X.reshape(2003, 15, X.shape[3])
y = abstraksi

In [36]:
X_train, y_train = X[train_index], y[train_index]
X_test, y_test = X[test_index], y[test_index]
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1702, 15, 5017) (1702, 6) (301, 15, 5017) (301, 6)


In [31]:
n = 5
encoder = encoder_cakapan

In [30]:
def run_tahap_4(X_train, y_train, X_test, y_test, n, encoder):
  model = None
  best_acc = 0
  for i in range(n):
    model, accuracy_score = train_stratified_random_sampling(X_train, y_train, X_test, y_test, X_train.shape[1:], y.shape[1], 3, encoder)
    if best_acc < accuracy_score:
      best_acc = accuracy_score
      best_model = model
  return best_model

In [39]:
def run_tahap_5_cv(X_train, y_train, n):
  performances = []
  for i in range(n):
    p = []
    loss_score, accuracy_score = train_5fold_cv(X_train, y_train, X_train.shape[1:], y.shape[1], 3)
    p.append((batch_size, loss_score, accuracy_score))
    performances.append((i, p))
  return performances

### Tanpa Resampling

In [44]:
callbacks = [EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.0001, patience=3, restore_best_weights=True)]
run_tahap_5_cv(X_train, y_train, 1)

Epoch 1/10
43/43 [==============================] - 6s 45ms/step - loss: 1.7264 - accuracy: 0.2879 - val_loss: 1.5238 - val_accuracy: 0.3343
Epoch 2/10
43/43 [==============================] - 1s 15ms/step - loss: 1.4134 - accuracy: 0.4021 - val_loss: 1.4645 - val_accuracy: 0.3314
Epoch 3/10
43/43 [==============================] - 1s 15ms/step - loss: 1.3107 - accuracy: 0.4415 - val_loss: 1.4233 - val_accuracy: 0.3695
Epoch 4/10
43/43 [==============================] - 1s 15ms/step - loss: 1.2254 - accuracy: 0.4824 - val_loss: 1.3652 - val_accuracy: 0.4194
Epoch 5/10
43/43 [==============================] - 1s 14ms/step - loss: 0.9368 - accuracy: 0.6701 - val_loss: 1.4281 - val_accuracy: 0.3930
Epoch 6/10
43/43 [==============================] - 1s 14ms/step - loss: 0.7578 - accuracy: 0.7509 - val_loss: 1.5679 - val_accuracy: 0.4516
Epoch 7/10
43/43 [==============================] - 1s 15ms/step - loss: 0.5041 - accuracy: 0.8623 - val_loss: 1.8013 - val_accuracy: 0.4487
Epoch 8/10
43

[(0, [(32, 1.8251, 0.4842)])]

In [37]:
best_model = run_tahap_4(X_train, y_train, X_test, y_test, n, encoder)

Epoch 1/10
54/54 [==============================] - 5s 36ms/step - loss: 1.6974 - accuracy: 0.3074 - val_loss: 1.4217 - val_accuracy: 0.3754
Epoch 2/10
54/54 [==============================] - 1s 14ms/step - loss: 1.4083 - accuracy: 0.3692 - val_loss: 1.3878 - val_accuracy: 0.3754
Epoch 3/10
10/10 [==============================] - 0s 7ms/step - loss: 1.4217 - accuracy: 0.3754
Loss Score :  1.4217 Accuracy Score :  0.3754
              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.0000    0.0000    0.0000        42
       Sedih     0.0000    0.0000    0.0000        86
      Senang     0.3754    1.0000    0.5459       113
       Takut     0.0000    0.0000    0.0000         9
    Terkejut     0.0000    0.0000    0.0000        50

    accuracy                         0.3754       301
   macro avg     0.0626    0.1667    0.0910       301
weighted avg     0.1409    0.3754    0.2049       301

[[  0   0   0   1   0   0]
 [  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
54/54 [==============================] - 6s 37ms/step - loss: 1.7186 - accuracy: 0.3497 - val_loss: 1.4612 - val_accuracy: 0.3754
Epoch 2/10
54/54 [==============================] - 1s 14ms/step - loss: 1.4395 - accuracy: 0.3721 - val_loss: 1.3961 - val_accuracy: 0.3754
Epoch 3/10
54/54 [==============================] - 1s 14ms/step - loss: 1.3527 - accuracy: 0.3873 - val_loss: 1.3119 - val_accuracy: 0.4286
Epoch 4/10
54/54 [==============================] - 1s 14ms/step - loss: 1.1493 - accuracy: 0.5005 - val_loss: 1.2722 - val_accuracy: 0.5249
Epoch 5/10
54/54 [==============================] - 1s 14ms/step - loss: 0.8746 - accuracy: 0.6693 - val_loss: 1.3869 - val_accuracy: 0.5116
Epoch 6/10
54/54 [==============================] - 1s 14ms/step - loss: 0.6414 - accuracy: 0.8122 - val_loss: 1.5838 - val_accuracy: 0.5316
Epoch 7/10
54/54 [==============================] - 1s 14ms/step - loss: 0.4480 - accuracy: 0.8795 - val_loss: 1.7208 - val_accuracy: 0.5249
Epoch 8/10
10

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
54/54 [==============================] - 5s 37ms/step - loss: 1.7044 - accuracy: 0.3240 - val_loss: 1.4341 - val_accuracy: 0.3754
Epoch 2/10
54/54 [==============================] - 1s 14ms/step - loss: 1.4462 - accuracy: 0.3637 - val_loss: 1.3889 - val_accuracy: 0.3754
Epoch 3/10
54/54 [==============================] - 1s 14ms/step - loss: 1.3587 - accuracy: 0.4235 - val_loss: 1.3334 - val_accuracy: 0.4352
Epoch 4/10
54/54 [==============================] - 1s 14ms/step - loss: 1.2165 - accuracy: 0.4867 - val_loss: 1.2658 - val_accuracy: 0.4585
Epoch 5/10
54/54 [==============================] - 1s 14ms/step - loss: 0.9423 - accuracy: 0.6200 - val_loss: 1.2597 - val_accuracy: 0.5017
Epoch 6/10
54/54 [==============================] - 1s 14ms/step - loss: 0.6811 - accuracy: 0.7867 - val_loss: 1.4398 - val_accuracy: 0.5249
Epoch 7/10
54/54 [==============================] - 1s 14ms/step - loss: 0.4575 - accuracy: 0.8771 - val_loss: 1.5731 - val_accuracy: 0.4884
Epoch 8/10
10

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
54/54 [==============================] - 5s 36ms/step - loss: 1.7045 - accuracy: 0.3113 - val_loss: 1.4301 - val_accuracy: 0.3754
Epoch 2/10
54/54 [==============================] - 1s 13ms/step - loss: 1.3986 - accuracy: 0.3804 - val_loss: 1.3835 - val_accuracy: 0.3754
Epoch 3/10
54/54 [==============================] - 1s 14ms/step - loss: 1.3348 - accuracy: 0.3904 - val_loss: 1.3126 - val_accuracy: 0.3987
Epoch 4/10
54/54 [==============================] - 1s 14ms/step - loss: 1.1578 - accuracy: 0.4962 - val_loss: 1.3077 - val_accuracy: 0.4850
Epoch 5/10
54/54 [==============================] - 1s 14ms/step - loss: 0.9179 - accuracy: 0.6797 - val_loss: 1.3041 - val_accuracy: 0.5017
Epoch 6/10
54/54 [==============================] - 1s 14ms/step - loss: 0.6765 - accuracy: 0.8059 - val_loss: 1.4206 - val_accuracy: 0.4518
Epoch 7/10
10/10 [==============================] - 0s 8ms/step - loss: 1.3041 - accuracy: 0.5017
Loss Score :  1.3041 Accuracy Score :  0.5017
          

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
54/54 [==============================] - 5s 36ms/step - loss: 1.7161 - accuracy: 0.3340 - val_loss: 1.4327 - val_accuracy: 0.3754
Epoch 2/10
54/54 [==============================] - 1s 14ms/step - loss: 1.4188 - accuracy: 0.3786 - val_loss: 1.3768 - val_accuracy: 0.4020
Epoch 3/10
54/54 [==============================] - 1s 14ms/step - loss: 1.3016 - accuracy: 0.4511 - val_loss: 1.3152 - val_accuracy: 0.4286
Epoch 4/10
54/54 [==============================] - 1s 14ms/step - loss: 1.1137 - accuracy: 0.5504 - val_loss: 1.2785 - val_accuracy: 0.4751
Epoch 5/10
54/54 [==============================] - 1s 14ms/step - loss: 0.8434 - accuracy: 0.6988 - val_loss: 1.3631 - val_accuracy: 0.5050
Epoch 6/10
54/54 [==============================] - 1s 14ms/step - loss: 0.5781 - accuracy: 0.8628 - val_loss: 1.5417 - val_accuracy: 0.5017
Epoch 7/10
54/54 [==============================] - 1s 14ms/step - loss: 0.3884 - accuracy: 0.9029 - val_loss: 1.7088 - val_accuracy: 0.5150
Epoch 8/10
54

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
best_model.save(drive_dir + "/Eksperimen/Model/model_leksikal_langsung_tes.h5")

### Dengan Resampling

In [26]:
loss_score = []
accuracy_score = []

cv = KFold(n_splits=5, random_state=42, shuffle=True)
i = 0
for train_index, val_index in cv.split(X):
    
    X_train, X_val, y_train, y_val = X[train_index], X[val_index], y[train_index], y[val_index]
    try:
      X_train_reshape = X_train.reshape((X_train.shape[0], X_train.shape[1]*X_train.shape[2]))
      oversample = SMOTE(kind='regular', k_neighbors=2)
      X_train_resampling, y_train_resampling = oversample.fit_resample(X_train_reshape, y_train)
      X_train_resampling = X_train_resampling.reshape(X_train_resampling.shape[0], 15, int(X_train_resampling.shape[1]/15))

      model = create_model(X_train_resampling.shape[1:], y_train_resampling.shape[1], 3)
      model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
      model.fit(X_train_resampling, y_train_resampling,
                validation_data=(X_val, y_val),
                epochs=epochs,
                batch_size=batch_size,
                callbacks=callbacks)
    except:
      model = create_model(X_train.shape[1:], y_train.shape[1], 3)
      model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
      model.fit(X_train, y_train,
              validation_data=(X_val, y_val),
              epochs=epochs,
              batch_size=batch_size,
              callbacks=callbacks)
        
    l, a = model.evaluate(X_val, y_val)
    loss_score.append(l)
    accuracy_score.append(a)
  
loss_score = np.round(np.mean(loss_score), 4)
accuracy_score = np.round(np.mean(accuracy_score), 4)
print("Loss Score : ", loss_score, "Accuracy Score : ", accuracy_score)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

Epoch 1/10
114/114 [==============================] - 25s 23ms/step - loss: 1.7084 - accuracy: 0.3757 - val_loss: 1.4657 - val_accuracy: 0.3691
Epoch 2/10
114/114 [==============================] - 1s 11ms/step - loss: 0.9567 - accuracy: 0.5883 - val_loss: 1.5070 - val_accuracy: 0.3890
Epoch 3/10
114/114 [==============================] - 1s 11ms/step - loss: 0.6484 - accuracy: 0.7632 - val_loss: 1.6520 - val_accuracy: 0.4214
Epoch 4/10
114/114 [==============================] - 1s 11ms/step - loss: 0.4697 - accuracy: 0.8414 - val_loss: 1.8529 - val_accuracy: 0.4464
Epoch 5/10
114/114 [==============================] - 1s 11ms/step - loss: 0.3023 - accuracy: 0.9085 - val_loss: 2.0628 - val_accuracy: 0.4289
Epoch 6/10
114/114 [==============================] - 1s 11ms/step - loss: 0.1760 - accuracy: 0.9561 - val_loss: 2.3491 - val_accuracy: 0.4539
Epoch 7/10
114/114 [==============================] - 1s 10ms/step - loss: 0.0959 - accuracy: 0.9810 - val_loss: 2.5110 - val_accuracy: 0.436

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

Epoch 1/10
115/115 [==============================] - 6s 23ms/step - loss: 1.6736 - accuracy: 0.3322 - val_loss: 1.5591 - val_accuracy: 0.3167
Epoch 2/10
115/115 [==============================] - 1s 12ms/step - loss: 0.9637 - accuracy: 0.5930 - val_loss: 1.5991 - val_accuracy: 0.3217
Epoch 3/10
115/115 [==============================] - 1s 12ms/step - loss: 0.7082 - accuracy: 0.7350 - val_loss: 1.6173 - val_accuracy: 0.3865
Epoch 4/10
115/115 [==============================] - 1s 12ms/step - loss: 0.4341 - accuracy: 0.8221 - val_loss: 1.7667 - val_accuracy: 0.4140
Epoch 5/10
115/115 [==============================] - 1s 12ms/step - loss: 0.3077 - accuracy: 0.8745 - val_loss: 1.9204 - val_accuracy: 0.4165
Epoch 6/10
115/115 [==============================] - 1s 12ms/step - loss: 0.2419 - accuracy: 0.9074 - val_loss: 2.0575 - val_accuracy: 0.4638
Epoch 7/10
115/115 [==============================] - 1s 12ms/step - loss: 0.1750 - accuracy: 0.9550 - val_loss: 2.2469 - val_accuracy: 0.4389

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Epoch 1/10
51/51 [==============================] - 6s 40ms/step - loss: 1.7022 - accuracy: 0.3509 - val_loss: 1.4977 - val_accuracy: 0.3815
Epoch 2/10
51/51 [==============================] - 1s 14ms/step - loss: 1.4119 - accuracy: 0.3726 - val_loss: 1.4457 - val_accuracy: 0.3815
Epoch 3/10
51/51 [==============================] - 1s 14ms/step - loss: 1.3460 - accuracy: 0.3947 - val_loss: 1.4165 - val_accuracy: 0.3890
Epoch 4/10
51/51 [==============================] - 1s 14ms/step - loss: 1.1657 - accuracy: 0.4738 - val_loss: 1.3811 - val_accuracy: 0.4713
Epoch 5/10
51/51 [==============================] - 1s 15ms/step - loss: 0.8438 - accuracy: 0.7035 - val_loss: 1.4422 - val_accuracy: 0.4389
Epoch 6/10
13/13 [==============================] - 0s 7ms/step - loss: 1.3811 - accuracy: 0.4713


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

Epoch 1/10
111/111 [==============================] - 6s 27ms/step - loss: 1.7090 - accuracy: 0.3124 - val_loss: 1.5610 - val_accuracy: 0.3050
Epoch 2/10
111/111 [==============================] - 1s 12ms/step - loss: 1.0789 - accuracy: 0.5821 - val_loss: 1.5134 - val_accuracy: 0.3250
Epoch 3/10
111/111 [==============================] - 1s 12ms/step - loss: 0.6744 - accuracy: 0.7388 - val_loss: 1.4772 - val_accuracy: 0.4600
Epoch 4/10
111/111 [==============================] - 1s 12ms/step - loss: 0.4321 - accuracy: 0.8654 - val_loss: 1.6666 - val_accuracy: 0.4650
Epoch 5/10
111/111 [==============================] - 1s 12ms/step - loss: 0.2668 - accuracy: 0.9246 - val_loss: 2.0069 - val_accuracy: 0.4600
Epoch 6/10
111/111 [==============================] - 1s 12ms/step - loss: 0.1416 - accuracy: 0.9666 - val_loss: 2.0562 - val_accuracy: 0.4925
Epoch 7/10
111/111 [==============================] - 1s 12ms/step - loss: 0.0789 - accuracy: 0.9845 - val_loss: 2.3115 - val_accuracy: 0.4650

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

Epoch 1/10
113/113 [==============================] - 6s 23ms/step - loss: 1.7013 - accuracy: 0.3792 - val_loss: 1.4686 - val_accuracy: 0.3875
Epoch 2/10
113/113 [==============================] - 1s 12ms/step - loss: 0.9162 - accuracy: 0.6161 - val_loss: 1.3769 - val_accuracy: 0.4325
Epoch 3/10
113/113 [==============================] - 1s 12ms/step - loss: 0.6000 - accuracy: 0.8266 - val_loss: 1.4147 - val_accuracy: 0.4975
Epoch 4/10
113/113 [==============================] - 1s 12ms/step - loss: 0.3240 - accuracy: 0.9146 - val_loss: 1.7034 - val_accuracy: 0.4800
Epoch 5/10
13/13 [==============================] - 0s 8ms/step - loss: 1.4147 - accuracy: 0.4975


In [27]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1]*X_train.shape[2]))
oversample = SMOTE(kind='regular', k_neighbors=2)
X_train_resampling, y_train_resampling = oversample.fit_resample(X_train, y_train)
X_train_resampling = X_train_resampling.reshape(3828, 15, 5017)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [34]:
best_model = run_tahap_4(X_train_resampling, y_train_resampling, X_test, y_test, 5, encoder)

Epoch 1/10
120/120 [==============================] - 6s 22ms/step - loss: 1.6933 - accuracy: 0.3499 - val_loss: 1.4672 - val_accuracy: 0.3422
Epoch 2/10
120/120 [==============================] - 1s 12ms/step - loss: 0.9582 - accuracy: 0.6347 - val_loss: 1.4887 - val_accuracy: 0.3023
Epoch 3/10
120/120 [==============================] - 1s 12ms/step - loss: 0.6188 - accuracy: 0.7408 - val_loss: 1.5732 - val_accuracy: 0.3854
Epoch 4/10
120/120 [==============================] - 1s 12ms/step - loss: 0.4258 - accuracy: 0.8422 - val_loss: 1.6479 - val_accuracy: 0.4551
Epoch 5/10
120/120 [==============================] - 1s 12ms/step - loss: 0.2626 - accuracy: 0.9042 - val_loss: 1.8498 - val_accuracy: 0.4651
Epoch 6/10
120/120 [==============================] - 1s 13ms/step - loss: 0.1819 - accuracy: 0.9431 - val_loss: 2.1689 - val_accuracy: 0.4585
Epoch 7/10
120/120 [==============================] - 1s 12ms/step - loss: 0.1180 - accuracy: 0.9672 - val_loss: 2.2948 - val_accuracy: 0.4817

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
120/120 [==============================] - 6s 21ms/step - loss: 1.6808 - accuracy: 0.3313 - val_loss: 1.6075 - val_accuracy: 0.2625
Epoch 2/10
120/120 [==============================] - 1s 12ms/step - loss: 0.9776 - accuracy: 0.6331 - val_loss: 1.4094 - val_accuracy: 0.4286
Epoch 3/10
120/120 [==============================] - 1s 12ms/step - loss: 0.6268 - accuracy: 0.7708 - val_loss: 1.5426 - val_accuracy: 0.4385
Epoch 4/10
120/120 [==============================] - 1s 12ms/step - loss: 0.4183 - accuracy: 0.8592 - val_loss: 1.6630 - val_accuracy: 0.4983
Epoch 5/10
120/120 [==============================] - 1s 12ms/step - loss: 0.2312 - accuracy: 0.9346 - val_loss: 2.1331 - val_accuracy: 0.4784
Epoch 6/10
10/10 [==============================] - 0s 7ms/step - loss: 1.6630 - accuracy: 0.4983
Loss Score :  1.663 Accuracy Score :  0.4983
              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.4706    0.1905

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
120/120 [==============================] - 7s 27ms/step - loss: 1.6736 - accuracy: 0.3837 - val_loss: 1.3960 - val_accuracy: 0.4086
Epoch 2/10
120/120 [==============================] - 1s 12ms/step - loss: 0.8718 - accuracy: 0.6585 - val_loss: 1.4514 - val_accuracy: 0.3621
Epoch 3/10
10/10 [==============================] - 0s 7ms/step - loss: 1.3960 - accuracy: 0.4086
Loss Score :  1.396 Accuracy Score :  0.4086
              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.3636    0.2857    0.3200        42
       Sedih     0.3452    0.3372    0.3412        86
      Senang     0.4425    0.6814    0.5366       113
       Takut     0.0000    0.0000    0.0000         9
    Terkejut     0.5000    0.1000    0.1667        50

    accuracy                         0.4086       301
   macro avg     0.2752    0.2341    0.2274       301
weighted avg     0.3986    0.4086    0.3713       301

[[ 0  0  1  0  0  0]
 [ 0 12

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
120/120 [==============================] - 6s 22ms/step - loss: 1.6838 - accuracy: 0.3290 - val_loss: 1.5870 - val_accuracy: 0.2757
Epoch 2/10
120/120 [==============================] - 1s 12ms/step - loss: 1.0821 - accuracy: 0.6138 - val_loss: 1.4289 - val_accuracy: 0.3787
Epoch 3/10
120/120 [==============================] - 1s 12ms/step - loss: 0.5744 - accuracy: 0.8014 - val_loss: 1.6010 - val_accuracy: 0.3887
Epoch 4/10
120/120 [==============================] - 1s 12ms/step - loss: 0.3745 - accuracy: 0.8861 - val_loss: 1.8688 - val_accuracy: 0.4186
Epoch 5/10
120/120 [==============================] - 1s 12ms/step - loss: 0.2200 - accuracy: 0.9411 - val_loss: 2.2642 - val_accuracy: 0.4286
Epoch 6/10
120/120 [==============================] - 1s 12ms/step - loss: 0.1593 - accuracy: 0.9566 - val_loss: 2.5346 - val_accuracy: 0.4419
Epoch 7/10
120/120 [==============================] - 1s 12ms/step - loss: 0.0928 - accuracy: 0.9799 - val_loss: 2.7690 - val_accuracy: 0.4153

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
120/120 [==============================] - 7s 22ms/step - loss: 1.6832 - accuracy: 0.3300 - val_loss: 1.5533 - val_accuracy: 0.3588
Epoch 2/10
120/120 [==============================] - 1s 12ms/step - loss: 1.0074 - accuracy: 0.6493 - val_loss: 1.5235 - val_accuracy: 0.3787
Epoch 3/10
120/120 [==============================] - 1s 12ms/step - loss: 0.5122 - accuracy: 0.8038 - val_loss: 1.5492 - val_accuracy: 0.4286
Epoch 4/10
120/120 [==============================] - 1s 12ms/step - loss: 0.3334 - accuracy: 0.8895 - val_loss: 1.6604 - val_accuracy: 0.4784
Epoch 5/10
120/120 [==============================] - 1s 12ms/step - loss: 0.2316 - accuracy: 0.9346 - val_loss: 1.8362 - val_accuracy: 0.4551
Epoch 6/10
10/10 [==============================] - 0s 7ms/step - loss: 1.6604 - accuracy: 0.4784
Loss Score :  1.6604 Accuracy Score :  0.4784
              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.4762    0.238

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Tahap 6 : Penyesuaian Design Eksperimen Untuk Meningkatkan Performa

### Delete Small Size Label Data

In [ ]:
def get_label_idxs(list, label):
  idxs = []
  for i in range(len(list)):
    for l in label:
      if list[i] == l:
        idxs.append(i)
  return np.array(idxs)

In [ ]:
dec_abstraksi = encoder_cakapan.inverse_transform(abstraksi)
del_idxs = get_label_idxs(dec_abstraksi, ["Jijik", "Takut"])
updated_features = np.delete(sequential_features, del_idxs)
updated_labels = np.delete(dec_abstraksi, del_idxs)
updated_labels = updated_labels.reshape(-1, 1)
print(updated_features.shape, updated_labels.shape)

(1936,) (1936, 1)


In [ ]:
new_encoder_cakapan = OneHotEncoder(sparse=False)
updated_labels = updated_labels.reshape(-1, 1)
updated_labels = new_encoder_cakapan.fit_transform(updated_labels)
updated_labels.shape

(1936, 4)

In [ ]:
X = tf.keras.preprocessing.sequence.pad_sequences(updated_features, padding="post", dtype='float32')
X = X.reshape(X.shape[0], X.shape[1], X.shape[3])
y = tf.keras.preprocessing.sequence.pad_sequences(updated_labels, padding="post", dtype='float32')
print(X.shape, y.shape)

(1936, 15, 5017) (1936, 4)


In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
for train_index, test_index in sss.split(X, y):
  train_index, test_index = train_index, test_index

In [ ]:
n = 5
encoder = new_encoder_cakapan

In [ ]:
best_model = run_tahap_4(X[train_index], y[train_index], X[test_index], y[test_index], n, new_encoder_cakapan)

Epoch 1/10
52/52 [==============================] - 6s 38ms/step - loss: 1.3591 - accuracy: 0.3664 - val_loss: 1.2884 - val_accuracy: 0.3883
Epoch 2/10
52/52 [==============================] - 1s 14ms/step - loss: 1.2594 - accuracy: 0.4030 - val_loss: 1.2308 - val_accuracy: 0.3952
Epoch 3/10
52/52 [==============================] - 1s 14ms/step - loss: 1.0848 - accuracy: 0.5137 - val_loss: 1.1579 - val_accuracy: 0.4639
Epoch 4/10
52/52 [==============================] - 1s 14ms/step - loss: 0.7766 - accuracy: 0.7308 - val_loss: 1.2497 - val_accuracy: 0.4708
Epoch 5/10
52/52 [==============================] - 1s 14ms/step - loss: 0.5025 - accuracy: 0.8672 - val_loss: 1.4054 - val_accuracy: 0.4708
Epoch 6/10
52/52 [==============================] - 1s 14ms/step - loss: 0.3181 - accuracy: 0.9358 - val_loss: 1.8195 - val_accuracy: 0.4777
Epoch 7/10
52/52 [==============================] - 1s 14ms/step - loss: 0.1965 - accuracy: 0.9625 - val_loss: 1.9227 - val_accuracy: 0.4502
Epoch 8/10
10

In [ ]:
best_model.save(drive_dir + "/Eksperimen/Model/model_leksikal_langsung_4.h5")

In [ ]:
model_leksikal = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_leksikal_langsung_4.h5")
model_leksikal.evaluate(X[test_index], y[test_index])

10/10 [==============================] - 1s 10ms/step - loss: 1.1101 - accuracy: 0.5498


[1.110121726989746, 0.5498281717300415]

In [ ]:
pred = model_leksikal.predict(X[test_index])
y_pred = new_encoder_cakapan.inverse_transform(pred)
y_true = new_encoder_cakapan.inverse_transform(y[test_index])

print(classification_report(y_true, y_pred, digits=4))
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

       Marah     0.5526    0.5000    0.5250        42
       Sedih     0.4375    0.4070    0.4217        86
      Senang     0.6202    0.7080    0.6612       113
    Terkejut     0.5455    0.4800    0.5106        50

    accuracy                         0.5498       291
   macro avg     0.5389    0.5237    0.5296       291
weighted avg     0.5436    0.5498    0.5449       291

[[21 13  5  3]
 [ 8 35 34  9]
 [ 3 22 80  8]
 [ 6 10 10 24]]
